NOTE: The *first* time that you start investigating a state, you need to load the data for it by using this notebook.
Once the data has been pulled to your local server, it can be re-used until the local database is purged.

In [ ]:
import emission.public.pull_and_load_public_data as plpd
import pandas as pd

In [ ]:
import emission.storage.timeseries.aggregate_timeseries as estag

In [ ]:
public_ids = estag.TEST_PHONE_IDS

### Pull based on state and regime

In [ ]:
ranges_df = pd.read_csv("../results_spring_2016/ranges.csv", error_bad_lines=False)

In [ ]:
ranges_df.head()

In [ ]:
regime_grouped_ranges_df = ranges_df.groupby(['state', 'android_regime', 'ios_regime'])

In [ ]:
def pull_data_from_server(state, android_regime):
    for key, df in regime_grouped_ranges_df:
        if key[0] == state and key[1] == android_regime:
            # one row for each phone
            # each row contains len(moving ranges) dataframes
            for row in df.to_records():
                print "Pulling data for range %s -> %s" % (row.start_ts, row.end_ts)
                plpd.request_batched_data("https://e-mission.eecs.berkeley.edu", row.start_ts, row.end_ts, 
                                          map(str, public_ids))

In [ ]:
pull_data_from_server("moving", "high+1sec")

In [ ]:
pull_data_from_server("stationary", "high+1sec")

In [ ]:
pull_data_from_server("loitering", "high+1sec")

### Pull for comparing with specified ground truth

In [ ]:
caltrain_df = pd.read_csv("../results_spring_2016/caltrain_trips.csv")

In [ ]:
caltrain_df

In [ ]:
# Pulling data for the caltrain ground truth comparison
for row in caltrain_df.to_records():
    print "Pulling data for range %s -> %s" % (row.start_ts, row.end_ts)
    plpd.request_batched_data("https://e-mission.eecs.berkeley.edu", row.start_ts, row.end_ts, map(str, public_ids))